1. Stage i is represted by year i,i 1,2....,n
2. The alternative at stage (year) i are keep (K) or replace (R) the machine at the start of year i
3. the state at stage i is the age of the machine at the start of year i

Given that the machine is t year old at the start of yaer i, define 
$$ 
    f_i(t) = \text{maximum net income for years } i,i+1,...., \text{ and n} 
$$

The recursive equation is:
$$
   f_n(t) = max
    \begin{cases}
        r(t)-c(t)+S(t+1) &{if} &{keep} \\
        r(t)+s(t)-S(1) -I-c(0)& {if}&{replace} 
    \end{cases}
$$

$$
   f_i(t) = max
    \begin{cases}
        y(t)-c(t)+f_{i+1}(t+1) &{if} &{keep} \\
        y(t)+s(t)-I -c(0)+f_{i+1}(1)& {if}&{replace} 
    \end{cases}
$$

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
#input values

#nodes
n = 4
start = 3
max_age = 6

#Machine inputs
r = [20000,19000,18500,17200,15500,14000,12200]
c = [200,600,1200,1500,1700,1800,2200]
s = [0,80000,60000,50000,30000,10000,5000]
I = 100000

#df
df_machine_values = pd.DataFrame()
df_machine_values["Age"] = [0,1,2,3,4,5,6]
df_machine_values["Revenue"] = [20000,19000,18500,17200,15500,14000,12200]
df_machine_values["Costs"] = [200,600,1200,1500,1700,1800,2200]
df_machine_values["Salvage_value"] = [0,80000,60000,50000,30000,10000,5000]


In [29]:
# functions
def nodes(n,start,max_age):
    #define array and start value
    node_tab = np.zeros((max_age,n+1))
    node_tab[start-1][0] = 1

    #find all nodes in the network 
    for i in range(0,n):
        for j in range(0,max_age-1):
            if node_tab[j][i] == 1:
                if j == max_age-1:
                    node_tab[0][i+1] == 1
                else:
                    node_tab[0][i+1] = 1
                    node_tab[j+1][i+1] =1
    return node_tab

# calculations 
def calucations(nodes,r,c,s,n,I,max_age):
    f_t = np.zeros((max_age,n))

    #Set up order for calucaltions
    decisonYear = list(range(n))
    decisonYear.reverse()

    #Chain for optimal decision
    chain = []
    
    # DataFrame to store all values
    df = pd.DataFrame(columns= ["Stage","State","Keep","Replace","opt_value","Decision"])

    #Cacluate stages and find the optimcal decision for each state for each decision year.
    for i in decisonYear:   #Start at the last stage of the life cycle
        if i == max(decisonYear):
            for j in range(0,max_age):  #loop trough all posible ages that the machines can have e.i  0 - max age
                if nodes[j][i] == 1:    #check if the age of the machine i possible
                    if j == max_age-1:
                        calc_Replace_temp = r[0] + s[j+1] + s[1] -c[0]-I
                        f_t[j][i] = calc_Replace_temp #optimum value 
                        df = df.append({"Stage":i+1,"State":j+1,"Replace": calc_Replace_temp,"opt_value":calc_Replace_temp,"Decision":"R"},ignore_index=True)
                    else:
                        calc_Keep_temp = r[j+1] + s[j+2]-c[j+1]
                        calc_Replace_temp = r[0] + s[j+1] + s[1] -c[0]-I
                        optimal_value = max(calc_Keep_temp,calc_Replace_temp)

                        f_t[j][i] = max(calc_Keep_temp,calc_Replace_temp) # optimum value

                        if calc_Keep_temp > calc_Replace_temp:
                            decision = "K"
                        elif calc_Keep_temp == calc_Replace_temp:
                            decision = "R or K"
                        else:
                            decision = "R"

                        df = df.append({"Stage":i+1,"State":j+1,"Keep":calc_Keep_temp, "Replace":calc_Replace_temp,"opt_value":optimal_value,"Decision":decision},ignore_index=True)           
        else:
            for j in range(0,max_age):
                if nodes[j][i] == 1:
                    calc_Keep_temp = r[j+1]-c[j+1]+f_t[j+1][i+1]
                    calc_Replace_temp = r[0] + s[j+1] - c[0] - I + f_t[0][i+1]
                    optimal_value = max(calc_Keep_temp,calc_Replace_temp)
                    f_t[j][i] = max(calc_Keep_temp,calc_Replace_temp) # optimum value

                    if calc_Keep_temp > calc_Replace_temp:
                        decision = "K"
                    elif calc_Keep_temp == calc_Replace_temp:
                        decision = "R or K"
                    else:
                        decision =" R"
                    df = df.append({"Stage":i+1,"State":j+1,"Keep":calc_Keep_temp, "Replace":calc_Replace_temp,"opt_value":optimal_value,"Decision":decision},ignore_index=True)
   

    #chain for optimal decisions
    for stage in range(1,max(df["Stage"]+1)):
        #For first stage
        if stage == 1:
            stages = df.loc[df["Stage"]== stage]
            age = stages["State"].values[0]
            opt_decision = stages["Decision"].item().strip()
            chain.append(opt_decision)
        else:
            if chain[-1] == "R":
                age = 1
                stages = df.loc[(df["Stage"]==stage) & (df["State"]== age)]
                opt_decision = stages["Decision"].item()
                chain.append(opt_decision)
            elif chain[-1] == "R or K": # forces to keep in order to retive one possible solution
                age  =+ 1
                stages = df.loc[(df["Stage"]==stage) & (df["State"]==age)]
                opt_decision = stages["Decision"].item()
                chain.append(opt_decision)
            else:
                age =+ 1
                stages = df.loc[(df["Stage"]==stage) & (df["State"]==age)]
                opt_decision = stages["Decision"].item()
                chain.append(opt_decision)
    
    return f_t, df, chain



In [31]:
#Solustions
allnodes = nodes(n,start,max_age)
f_t, df_solution, chain  = calucations(allnodes,r,c,s,n,I,max_age)

print("all possible nodes")
print(allnodes)
print("calculations")
print(df_solution.head())
print("Decision chain")
print(chain)

all possible nodes
[[0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [1. 0. 0. 1. 1.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]]
calculations
  Stage State     Keep  Replace opt_value Decision
0     4     1    78400    79800     79800        R
1     4     2    67300    59800     67300        K
2     4     3    45700    49800     49800        R
3     4     6      NaN     4800      4800        R
4     3     1  85700.0  79600.0   85700.0        K
Decision chain
['R', 'R or K', 'K', 'R']
